In [1]:
'''
Author: Zhenkun Shi
Date: 2022-11-18 02:05:01
LastEditors: Zhenkun Shi
LastEditTime: 2022-11-18 02:41:21
FilePath: /DMLF/baselines.ipynb
Description: 

Copyright (c) 2022 by tibd, All Rights Reserved. 
'''
import pandas as pd
import numpy as np
import joblib, os,time, argparse
import benchmark_common as bcommon
import config as cfg
import benchmark_test as btest
import benchmark_train as btrain
import benchmark_evaluation as eva
import tools.funclib as funclib
import tools.embedding_esm as esmebd
from tqdm import tqdm

from sklearn import metrics
from sklearn.model_selection import train_test_split
from gc import callbacks
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() 

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
print('step 1 loading task data')

data_task1_train = pd.read_feather(cfg.FILE_TASK1_TRAIN)
data_task2_train = pd.read_feather(cfg.FILE_TASK2_TRAIN)
data_task3_train = pd.read_feather(cfg.FILE_TASK3_TRAIN)

data_task1_test = pd.read_feather(cfg.FILE_TASK1_TEST_2022)
data_task2_test = pd.read_feather(cfg.FILE_TASK2_TEST_2022)
data_task3_test = pd.read_feather(cfg.FILE_TASK3_TEST_2022)

step 1 loading task data


In [3]:
data_task3_test

,id,seq,ec_number
0,P9WG69,MTTSVIVAGARTPIGKLMGSLKDFSASELGAIAIKGALEKANVPAS...,2.3.1.9
1,P9WI52,MAERAPRGEVAVMVAVQSALVDRPGMLATARGLSHFGEHCIGWLIL...,3.1.3.-
2,P9WQ38,MRALHVPAGSATALLLPALQRVLGGSDPALVAVPTQHESLLGALRV...,6.2.1.26
3,P9WIA9,DAGVSWKVYRNKTLGPISSVLTYGSLVTSFKQSADPRSDLVRFGVA...,3.1.4.3
4,D8GR70,MASYLTLFISAVVVNNYVLTRFLGLCIFFGVSKNLNASVGMGMAVT...,7.1.1.-
...,...,...,...
5106,A2WLP4,MVSAAAGWAAPAFAVAAVVIWVVLCGELLRRRRRGAGSGKGDAAAA...,1.14.14.-
5107,A0A250YGJ5,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVRQSSN...,"2.3.1.286,2.4.2.-"
5108,A0A2K5TU92,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVWQSSH...,"2.3.1.286,2.4.2.-"
5109,A0A3R0A696,MKHWKKMAASLIAISTMVAVVPTTYAMESEDSQPQTTDTATVQTTK...,3.2.1.55


## 1. DIAMOND

In [4]:
diamond_task1 = funclib.getblast(train=data_task1_train[['id','seq']], test=data_task1_test[['id','seq']])

res_task1_diamond =diamond_task1[['id','sseqid']].merge(data_task1_train[['id','isenzyme']], how='left', left_on='sseqid', right_on='id')
res_task1_diamond = res_task1_diamond[['id_x', 'isenzyme']].rename(columns={'id_x':'id','isenzyme':'isenzyme_pred'})
res_task1_diamond = data_task1_test.merge(res_task1_diamond, how='left', on='id')
res_task1_diamond['isenzyme_pred_full']=res_task1_diamond.apply(lambda x : x.isenzyme_pred if (str(x.isenzyme_pred)!='nan') else ( True if x.isenzyme==False else False)  , axis=1)
print('task1:\n----------------')
print('baslineName', '\t\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix(groundtruth=res_task1_diamond.isenzyme, predict=res_task1_diamond.isenzyme_pred_full, baselineName='Diamond', type='binary')

print('\ntask2:\n----------------')

diamond_task2 = funclib.getblast(train=data_task2_train[['id','seq']], test=data_task2_test[['id','seq']])
res_task2_diamond = diamond_task2[['id','sseqid']].merge(data_task2_train[['id','functionCounts']], how='left', left_on='sseqid', right_on='id')
res_task2_diamond = res_task2_diamond[['id_x', 'functionCounts']].rename(columns={'id_x':'id','functionCounts':'functionCounts_diamond'})
res_task2_diamond = data_task2_test.merge( res_task2_diamond, on='id', how='left')
res_task2_diamond = res_task2_diamond.fillna(-1)
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_task2_diamond.functionCounts, predict=res_task2_diamond.functionCounts_diamond, baselineName='Diamond', type='multi')

diamond_task3 = funclib.getblast(train=data_task3_train[['id','seq']], test=data_task3_test[['id','seq']])
res_task3_diamond = diamond_task3[['id','sseqid']].merge(data_task3_train[['id','ec_number']], how='left', left_on='sseqid', right_on='id')
res_task3_diamond = res_task3_diamond[['id_x', 'ec_number']].rename(columns={'id_x':'id','ec_number':'ec_number_diamond'})

res_task3_diamond = data_task3_test.merge( res_task3_diamond, on='id', how='left')
res_task3_diamond = res_task3_diamond.fillna('-')
print('\ntask3:\n----------------')
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_task3_diamond.ec_number, predict=res_task3_diamond.ec_number_diamond, baselineName='Diamond', type='multi')


Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet
task1:
----------------
baslineName 		 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 	 confusion Matrix
Diamond 		0.677030 	0.629801 		0.751150 	0.798865 	0.704330 	 tp: 4083 fp: 2400 fn: 1028 tn: 3103

task2:
----------------
Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
     Diamond  		0.744277  	0.434697 		0.350342 	0.273826
Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet

task3:
----------------
 baslineName 		

## 2. ECPred

In [37]:
res_ecpred = pd.read_csv(f'{cfg.RESULTSDIR}ecpred/ecpred.txt', sep='\t', header=0)
res_ecpred['isemzyme_ecpred'] = ''
with pd.option_context('mode.chained_assignment', None):
    res_ecpred.isemzyme_ecpred[res_ecpred['EC Number']=='non Enzyme'] = False
    res_ecpred.isemzyme_ecpred[res_ecpred['EC Number']!='non Enzyme'] = True
    
res_ecpred.columns = ['id','ec_ecpred', 'conf', 'isemzyme_ecpred']
res_ecpred = res_ecpred.iloc[:,np.r_[0,1,3]]
res_ecpred['functionCounts_ecpred'] = res_ecpred.ec_ecpred.apply(lambda x :len(str(x).split(',')))


In [51]:
res_task1_ecpred = data_task1_test.merge(res_ecpred, how='left', on='id')
res_task1_ecpred = res_task1_ecpred.drop_duplicates().reset_index(drop=True)
res_task1_ecpred.isemzyme_ecpred = res_task1_ecpred.apply(lambda x: x.isemzyme_ecpred if str(x.isemzyme_ecpred)!='nan'  else (True if x.isenzyme == False else False), axis=1)
res_task1_ecpred.isemzyme_ecpred = res_task1_ecpred.apply(lambda x: x.isemzyme_ecpred if x.ec_ecpred!='no Prediction' else (True if x.isenzyme == False else False), axis=1)
print('task1:\n----------------')
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix(groundtruth=res_task1_ecpred.isenzyme, predict=res_task1_ecpred.isemzyme_ecpred, baselineName='ecpred', type='binary')




print('\ntask2:\n----------------')


res_task2_ecpred = data_task2_test.merge(res_ecpred, on='id', how='left')
res_task2_ecpred = res_task2_ecpred.drop_duplicates().reset_index(drop=True)
res_task2_ecpred = res_task2_ecpred.fillna(-1)
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_task2_ecpred.functionCounts, predict=res_task2_ecpred.functionCounts_ecpred, baselineName='ecpred', type='multi')




res_task3_ecpred = data_task3_test.merge(res_ecpred, on='id', how='left')
res_task3_ecpred = res_task3_ecpred.drop_duplicates().reset_index(drop=True)
res_task3_ecpred = res_task3_ecpred.fillna('-')
print('\ntask3:\n----------------')
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_task3_ecpred.ec_number, predict=res_task3_ecpred.ec_ecpred, baselineName='ecpred', type='multi')

task1:
----------------
baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 	 confusion Matrix
ecpred 		0.802054 	0.752517 		0.865492 	0.877519 	0.810225 	 tp: 4485 fp: 1475 fn: 626 tn: 4028

task2:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      ecpred  		0.906868  	0.878957 		0.221721 	0.105684

task3:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      ecpred  		0.234983  	0.855199 		0.162045 	0.082026


In [43]:
res_task1_ecpred

,id,seq,isenzyme,ec_ecpred,isemzyme_ecpred,functionCounts_ecpred
0,P63153,MRGREFPLVLLALVLCQAPRGPAAPVSVGGGTVLAKMYPRGNHWAV...,False,no Prediction,True,1.0
1,P62861,MQLFVRAQELHTFEVTGQETVAQIKAHVASLEGIAPEDQVVLLAGA...,False,non Enzyme,False,1.0
2,P69031,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,False,non Enzyme,False,1.0
3,P69019,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,False,non Enzyme,False,1.0
4,Q5RF96,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,False,3.4.-.-,True,1.0
...,...,...,...,...,...,...
10609,A0A2K5TU92,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVWQSSH...,True,3.5.1.-,True,1.0
10610,A0A3R0A696,MKHWKKMAASLIAISTMVAVVPTTYAMESEDSQPQTTDTATVQTTK...,True,3.2.1.-,True,1.0
10611,Q5ZV91,MDEIKKDDELSQWLSTYGTITAERILGRYNISLPQDEILEAINIPS...,False,non Enzyme,False,1.0
10612,M1H607,MDAIKKKMQAMKLEKDDAMDRADTLEQQNKEANIRAEKAEEEVHNL...,False,non Enzyme,False,1.0


In [48]:
1475-21

1454

## 3. DeepEC

In [58]:
# !conda activate deepec
# !python ./baselines/deepec/deepec.py -i ./datasets/task1/test.fasta -o ./results/deepec/

res_deepec = pd.read_csv(cfg.FILE_DEEPEC_RESULTS, sep='\t',names=['id', 'ec_number'], header=0 )
res_deepec.ec_number=res_deepec.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
res_deepec.columns = ['id','ec_deepec']

res = []
for index, group in  res_deepec.groupby('id'):
    if len(group)==1:
        res = res + [[group.id.values[0], group.ec_deepec.values[0]]]
    else:
        ecs_str = ','.join(group.ec_deepec.values)
        res = res +[[group.id.values[0],ecs_str]] 
res_deepec = pd.DataFrame(res, columns=['id', 'ec_deepec'])


res_deepec_task1=data_task1_test.merge(res_deepec, on='id', how='left')
res_deepec_task1['isenzyme_pred']=res_deepec_task1.ec_deepec.apply(lambda x: True if str(x)!='nan' else False)

print('task1:\n----------------')
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix(groundtruth=res_deepec_task1.isenzyme, predict=res_deepec_task1.isenzyme_pred, baselineName='deepec', type='binary')

print('\ntask2:\n----------------')

res_deepec_task2=data_task2_test.merge(res_deepec, on='id', how='left')
res_deepec_task2=res_deepec_task2.fillna('-')
res_deepec_task2['functionCounts_deepec'] =res_deepec_task2.ec_deepec.apply(lambda x: len(x.split(',')))

print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_deepec_task2.functionCounts, predict=res_deepec_task2.functionCounts_deepec, baselineName='deepec', type='multi')

print('\ntask3:\n----------------')
res_deepec_task3 = data_task3_test.merge(res_deepec, on='id', how='left')
res_deepec_task3=res_deepec_task3.fillna('-')
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_deepec_task3.ec_number, predict=res_deepec_task3.ec_deepec, baselineName='deepec', type='multi')

task1:
----------------
baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 	 confusion Matrix
deepec 		0.638308 	0.944134 		0.590612 	0.264527 	0.413266 	 tp: 1352 fp: 80 fn: 3759 tn: 5423

task2:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      deepec  		0.906281  	0.817691 		0.135362 	0.138948

task3:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      deepec  		0.104872  	0.791039 		0.287901 	0.121086


## 4. CatFam

In [235]:
res_catfam = pd.read_csv(cfg.RESULTSDIR+'catfam/catfam.txt', sep='\t', names=['id', 'ec_catfam'])
res = []
for index, group in  res_catfam.groupby('id'):
    if len(group)==1:
        res = res + [[group.id.values[0], group.ec_catfam.values[0]]]
    else:
        ecs_str = ','.join(group.ec_catfam.values)
        res = res +[[group.id.values[0],ecs_str]] 
res_catfam = pd.DataFrame(res, columns=['id', 'ec_catfam'])
res_catfam = res_catfam.fillna('-')
res_catfam['isenzyme_catfam']=res_catfam.ec_catfam.apply(lambda x: True if str(x)!='-' else False)
res_catfam['functionCounts_catfam'] = res_catfam.ec_catfam.apply(lambda x :len(str(x).split(',')))

print('task1:\n----------------')
res_catfam_task1=data_task1_test.merge(res_catfam, on='id', how='left')
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix(groundtruth=res_catfam_task1.isenzyme, predict=res_catfam_task1.isenzyme_catfam, baselineName='catfam', type='binary')

print('\ntask2:\n----------------')
res_catfam_task2=data_task2_test.merge(res_catfam, on='id', how='left')
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_catfam_task2.functionCounts, predict=res_catfam_task2.functionCounts_catfam, baselineName='catfam', type='multi')

print('\ntask3:\n----------------')
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
res_catfam_task3=data_task3_test.merge(res_catfam, on='id', how='left')
eva.caculateMetrix(groundtruth=res_catfam_task3.ec_number, predict=res_catfam_task3.ec_catfam, baselineName='catfam', type='multi')

task1:
----------------
baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 	 confusion Matrix
catfam 		0.594404 	0.927813 		0.561931 	0.171004 	0.288782 	 tp: 874 fp: 68 fn: 4237 tn: 5435

task2:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      catfam  		0.910194  	0.662225 		0.159200 	0.174859

task3:
----------------
      catfam  		0.084328  	0.885812 		0.191622 	0.101330


## 5. PRIAM

In [60]:
res_priam = eva.load_praim_res(resfile=cfg.RESULTSDIR+'priam/PRIAM_20221011103347/ANNOTATION/sequenceECs.txt')


res_priam['isenzyme_priam'] = res_priam.ec_priam.apply(lambda x: True if str(x)!='nan' else False)
res_priam['functionCounts_priam'] = res_priam.ec_priam.apply(lambda x :len(str(x).split(',')))

print('task1:\n----------------')
res_priam_task1=data_task1_test.merge(res_priam, on='id', how='left')
# res_priam_task1['isenzyme_priam']=res_priam_task1.apply(lambda x: x.isenzyme_priam if x.isenzyme_priam==True else (False if x.isenzyme else True), axis=1)
res_priam_task1.isenzyme_priam = res_priam_task1.isenzyme_priam.fillna(False)
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix(groundtruth=res_priam_task1.isenzyme, predict=res_priam_task1.isenzyme_priam, baselineName='priam', type='binary')

print('\ntask2:\n----------------')
res_priam_task2=data_task2_test.merge(res_priam, on='id', how='left')
res_priam_task2.functionCounts_priam = res_priam_task2.functionCounts_priam.fillna(0)
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_priam_task2.functionCounts, predict=res_priam_task2.functionCounts_priam, baselineName='priam', type='multi')

print('\ntask3:\n----------------')
res_priam_task3=data_task3_test.merge(res_priam, on='id', how='left')
res_priam_task3.ec_priam = res_priam_task3.ec_priam.fillna('-')
print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
eva.caculateMetrix(groundtruth=res_priam_task3.ec_number, predict=res_priam_task3.ec_priam, baselineName='priam', type='multi')


task1:
----------------
baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 	 confusion Matrix
priam 		0.747315 	0.678001 		0.872065 	0.905107 	0.775264 	 tp: 4626 fp: 2197 fn: 485 tn: 3306

task2:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
       priam  		0.132655  	0.009457 		0.930938 	0.003176

task3:
----------------
 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
       priam  		0.045197  	0.250324 		0.752550 	0.014136


In [ ]:
java -jar /home/shizhenkun/codebase/DMLF/baselines/ECPred/ECPred.jar blast /home/shizhenkun/codebase/DMLF/data/datasets/task1/test.fasta /home/shizhenkun/codebase/DMLF/baselines/ECPred/ /home/shizhenkun/codebase/DMLF/results/ecpred

java -Xmx128G -jar /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_search.jar -p /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_JAN18 -i /home/shizhenkun/codebase/DMLF/data/datasets/task1/test.fasta -o /home/shizhenkun/codebase/DMLF/results/priam --blast_path /home/shizhenkun/downloads/blast-2.2.13/bin -np 100

In [3]:
EMBEDDING_METHOD = 'esm32'
print('step 1 loading task data')

data_task1_train = pd.read_feather(cfg.FILE_TASK1_TRAIN)
data_task2_train = pd.read_feather(cfg.FILE_TASK2_TRAIN)
print(f'step 2: Loading features, embdding method={EMBEDDING_METHOD}')
feature_df = bcommon.load_data_embedding(embedding_type=EMBEDDING_METHOD)

step 1 loading task data
step 2: Loading features, embdding method=esm32


In [4]:
print('step 3: train isEnzyme model')
task1_train_X, task1_train_Y = btrain.get_train_X_Y(traindata=data_task1_train, feature_bankfile=feature_df, task=1)
task1_test_X, task1_test_Y = btrain.get_train_X_Y(traindata=data_task1_test, feature_bankfile=feature_df, task=1)
t1_x_train, t1_x_vali, t1_y_train, t1_y_vali = train_test_split(task1_train_X,np.array(task1_train_Y).ravel(),test_size=cfg.VALIDATION_RATE,random_state=1)
t1_eval_set = [(t1_x_train, t1_y_train), (t1_x_vali, t1_y_vali), (task1_test_X, task1_test_Y)]

step 3: train isEnzyme model


In [15]:
cfg.TRAIN_ISENZYME_LEARNING_STEPS = 1000



model = XGBClassifier(
    objective='binary:logistic', 
    random_state=13, 
    use_label_encoder=False, 
    n_jobs=-2, 
    eval_metric='auc',
    max_depth=6,
    n_estimators= cfg.TRAIN_ISENZYME_LEARNING_STEPS,
    tree_method = 'gpu_hist',
    learning_rate = 0.01,
    gpu_id=0
    )

# print(model)
# model.fit(X, Y.ravel())
history = model.fit(t1_x_train, t1_y_train,  eval_set=t1_eval_set, verbose=100)

[0]	validation_0-auc:0.84186	validation_1-auc:0.84140	validation_2-auc:0.86341
[100]	validation_0-auc:0.88185	validation_1-auc:0.88011	validation_2-auc:0.89027
[200]	validation_0-auc:0.89308	validation_1-auc:0.89064	validation_2-auc:0.89717
[300]	validation_0-auc:0.90176	validation_1-auc:0.89837	validation_2-auc:0.89980
[400]	validation_0-auc:0.90862	validation_1-auc:0.90446	validation_2-auc:0.90188
[500]	validation_0-auc:0.91398	validation_1-auc:0.90909	validation_2-auc:0.90341
[600]	validation_0-auc:0.91817	validation_1-auc:0.91262	validation_2-auc:0.90431
[700]	validation_0-auc:0.92168	validation_1-auc:0.91545	validation_2-auc:0.90477
[800]	validation_0-auc:0.92471	validation_1-auc:0.91784	validation_2-auc:0.90520
[900]	validation_0-auc:0.92737	validation_1-auc:0.91989	validation_2-auc:0.90554
[999]	validation_0-auc:0.92986	validation_1-auc:0.92179	validation_2-auc:0.90590


In [5]:
methods=[ 'gbdt']
for method in methods:
    funclib.evaluate_2(baslineName=method, X_train_std=t1_x_train, Y_train=t1_y_train, X_test_std=task1_test_X, Y_test=task1_test_Y, type='binary')

gbdt 		0.820332 	0.822984 		0.818005 	0.798670 	0.810644 	 tp: 4082 fp: 878 fn: 1029 tn: 4625


In [12]:
task1_test_Y.values

array([0, 0, 0, ..., 0, 0, 1])

In [87]:
sprot2022 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2022.tsv', sep='\t',header=0) #读入文件
eclist = list(set(_flatten([item.split(',') for item in list(set(sprot2022.ec_number))])))

In [90]:
len(set([item.strip() for item in eclist]))

5570

In [6]:
from tkinter import _flatten

In [67]:
def get_first_digit(eclist):
    res =  list(set([item.split('.')[0] for item in eclist]))
    if len(res)==1:
        return res[0]
    else:
        return ','.join(res)

data_task3_train = pd.read_feather(cfg.FILE_TASK3_TRAIN)
data_task3_test_22 = pd.read_feather(cfg.FILE_TASK3_TEST_2022)
data_task3_test_20 = pd.read_feather(cfg.FILE_TASK3_TEST_2020)


data_task3_train['ecl1']=data_task3_train.ec_number.apply(lambda x : x.split('.')[0] if len(list(set(x.split(','))))==1 else get_first_digit(list(set(x.split(',')))))
data_task3_test_20['ecl1']=data_task3_test_20.ec_number.apply(lambda x : x.split('.')[0] if len(list(set(x.split(','))))==1 else get_first_digit(list(set(x.split(',')))))
data_task3_test_22['ecl1']=data_task3_test_22.ec_number.apply(lambda x : x.split('.')[0] if len(list(set(x.split(','))))==1 else get_first_digit(list(set(x.split(',')))))


In [68]:
for i in range(1,8):
    i_t = len(data_task3_train[data_task3_train.ecl1.str.contains(str(i))])
    i_20 = len(data_task3_test_20[data_task3_test_20.ecl1.str.contains(str(i))])
    i_22 = len(data_task3_test_22[data_task3_test_22.ecl1.str.contains(str(i))])
    
    print(f'{i_t}\t{i_20}\t{i_22}')

34169	1065	1615
79570	1009	1567
56749	676	989
20747	346	608
11927	108	184
25254	156	244
0	57	83


In [70]:

ec_task3_train = _flatten([item.split(',') for item in list(set(data_task3_train.ec_number))])
ec_task3_test_20 = _flatten([item.split(',') for item in list(set(data_task3_test_20.ec_number))])
ec_task3_test_22 = _flatten([item.split(',') for item in list(set(data_task3_test_22.ec_number))])



In [74]:
len(ec_task3_train)

6715

In [82]:
pd.DataFrame({'ec':ec_task3_train}).to_csv(cfg.RESULTSDIR+'ec_train.tsv', index=None, header=None)
pd.DataFrame({'ec':ec_task3_test_20}).to_csv(cfg.RESULTSDIR+'ec_test20.tsv', index=None, header=None)
pd.DataFrame({'ec':ec_task3_test_22}).to_csv(cfg.RESULTSDIR+'ec_test22.tsv', index=None, header=None)
